In [1]:
import os
import math
import pandas as pd

import matplotlib.pyplot as plt

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
notebook_path = os.path.abspath("explore_data.ipynb")
notebook_dir = os.path.dirname(notebook_path)

project_root = os.path.dirname(os.path.dirname(notebook_dir))
os.chdir(project_root)

In [3]:
store_loc_df = pd.read_excel("./data/Logistic Details_JH_Last 15 Days.xlsx", sheet_name="Store Lat Long")
wh_loc_df = pd.read_excel("./data/Logistic Details_JH_Last 15 Days.xlsx", sheet_name="WH Lat Long")
vehicles_df = pd.read_excel("./data/Logistic Details_JH_Last 15 Days.xlsx", sheet_name="Vehicles")
demand_df = pd.read_excel("./data/Logistic Details_JH_Last 15 Days.xlsx", sheet_name="Last 15 Days Demand")

In [4]:
# ============== Haversine Distance ==============
def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Compute Haversine distance in km between two (lat, lon) points."""
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlam = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlam/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def parse_lat_long(s):
    """Parse 'lat,lon' string to (lat, lon) tuple."""
    parts = str(s).strip().split(",")
    return float(parts[0]), float(parts[1])

In [5]:
# ============== Prepare Data for One Day ==============
# Pick a single delivery date
TARGET_DATE = pd.to_datetime("2026-01-22").date()
MAX_STORES = None  # Set to e.g. 20 for faster testing with fewer stores
day_demand = demand_df[demand_df["delivery_date"].dt.date == TARGET_DATE].copy()

# Aggregate demand by store (weight in kg)
store_demand = day_demand.groupby("store_name").agg({
    "Weight (in kg)": "sum",
    "start_time": "first",
    "end_time": "first",
    "city": "first"
}).reset_index()

# Build store name -> (lat, lon) mapping
store_coords = {}
for _, row in store_loc_df.iterrows():
    store_coords[row["store_name"]] = parse_lat_long(row["Lat Long"])

# Warehouse coords
wh_row = wh_loc_df.iloc[0]
wh_name = wh_row["WH Name"]
wh_coords = parse_lat_long(wh_row["Lat Long"])

# Filter to stores that have coords and demand
stores_with_demand = store_demand[store_demand["store_name"].isin(store_coords)].copy()
if MAX_STORES:
    stores_with_demand = stores_with_demand.head(MAX_STORES)
print(f"Date: {TARGET_DATE}")
print(f"Stores with demand: {len(stores_with_demand)}")
print(f"Total demand (kg): {stores_with_demand['Weight (in kg)'].sum():.1f}")

Date: 2026-01-22
Stores with demand: 73
Total demand (kg): 27350.8


In [6]:
# ============== Build Distance/Time Matrix & Time Windows ==============
# Nodes: 0 = depot (warehouse), 1..n = stores
AVG_SPEED_KMH = 30  # Average speed for travel time
SERVICE_TIME_MIN = 15  # Minutes to unload at each store

def time_to_minutes(t):
    """Convert datetime time to minutes since midnight."""
    if hasattr(t, "hour"):
        return t.hour * 60 + t.minute
    return 0

# Build locations list: [depot, store1, store2, ...]
locations = [wh_coords]
store_names = ["DEPOT"]
store_cities = ["DEPOT"]  # City for each node (for Adhoc vehicle restrictions)
for _, row in stores_with_demand.iterrows():
    locations.append(store_coords[row["store_name"]])
    store_names.append(row["store_name"])
    store_cities.append(row["city"])

n = len(locations)

# Haversine distance matrix (km)
dist_km = [[0.0] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if i != j:
            dist_km[i][j] = haversine_km(
                locations[i][0], locations[i][1],
                locations[j][0], locations[j][1]
            )

# Time matrix: travel time in minutes (distance/speed * 60) + service time at destination
# OR-Tools uses integer transit times
time_matrix = []
for i in range(n):
    row = []
    for j in range(n):
        if i == j:
            row.append(0)
        else:
            travel_min = int((dist_km[i][j] / AVG_SPEED_KMH) * 60)
            service = SERVICE_TIME_MIN if j > 0 else 0  # No service at depot
            row.append(travel_min + service)
    time_matrix.append(row)

# Time windows (minutes from midnight)
# Depot: 6 AM - 6 PM (360 - 1080)
# Stores: from demand data
time_windows = [(360, 1080)]  # Depot: 6am-6pm
for _, row in stores_with_demand.iterrows():
    start_min = time_to_minutes(row["start_time"])
    end_min = time_to_minutes(row["end_time"])
    time_windows.append((start_min, end_min))

# Demands (kg) - depot has 0
demands = [0] + stores_with_demand["Weight (in kg)"].astype(int).tolist()

In [7]:
# ============== Vehicle Setup (Fixed + Adhoc) ==============
ADHOC_PER_CITY = 5  # Pool of Adhoc vehicles per city (on-demand)
ADHOC_FIXED_COST = 50000  # Penalty so optimizer prefers Fixed vehicles

fixed_vehicles = vehicles_df[vehicles_df["Type"] == "Fixed"].dropna(subset=["Vehicle Count"])
adhoc_vehicles = vehicles_df[vehicles_df["Type"] == "Adhoc"]

vehicle_capacities = []
vehicle_types = []  # "Fixed" or ("Adhoc", city)
for _, row in fixed_vehicles.iterrows():
    count = int(row["Vehicle Count"])
    cap_kg = int(row["Weight Capacity in KG"])
    vehicle_capacities.extend([cap_kg] * count)
    vehicle_types.extend(["Fixed"] * count)

for _, row in adhoc_vehicles.iterrows():
    cap_kg = int(row["Weight Capacity in KG"])
    city = row["Location"]
    for _ in range(ADHOC_PER_CITY):
        vehicle_capacities.append(cap_kg)
        vehicle_types.append(("Adhoc", city))

num_fixed = sum(1 for t in vehicle_types if t == "Fixed")
num_adhoc = len(vehicle_types) - num_fixed
print(f"Fixed vehicles: {num_fixed}, Adhoc vehicles: {num_adhoc}")

Fixed vehicles: 18, Adhoc vehicles: 15


In [8]:
# ============== Build and Solve VRP with OR-Tools ==============
def solve_vrp(time_matrix, time_windows, demands, vehicle_capacities, vehicle_types,
              store_cities, depot=0, adhoc_fixed_cost=50000):
    """Solve CVRP with time windows. Fixed vehicles preferred; Adhoc used only when needed."""
    num_vehicles = len(vehicle_capacities)
    manager = pywrapcp.RoutingIndexManager(len(time_matrix), num_vehicles, depot)
    routing = pywrapcp.RoutingModel(manager)

    # Time callback (transit time between nodes)
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_matrix[from_node][to_node]

    transit_cb_idx = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb_idx)

    # Adhoc vehicles: high fixed cost so optimizer prefers Fixed vehicles
    # (SetAllowedVehiclesForIndex has a Python binding issue in this OR-Tools version,
    #  so Adhoc can visit any store; the cost penalty still minimizes their use)
    for v in range(num_vehicles):
        if vehicle_types[v] != "Fixed":
            routing.SetFixedCostOfVehicle(adhoc_fixed_cost, v)

    # Time dimension
    routing.AddDimension(
        transit_cb_idx, 60, 12 * 60, False, "Time"
    )
    time_dim = routing.GetDimensionOrDie("Time")
    for i, tw in enumerate(time_windows):
        idx = manager.NodeToIndex(i)
        time_dim.CumulVar(idx).SetRange(tw[0], tw[1])

    # Capacity dimension
    def demand_callback(from_index):
        return demands[manager.IndexToNode(from_index)]
    demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_cb_idx, 0, vehicle_capacities, True, "Capacity"
    )

    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_params.time_limit.seconds = 30

    solution = routing.SolveWithParameters(search_params)
    return solution, routing, manager, time_dim

solution, routing, manager, time_dim = solve_vrp(
    time_matrix, time_windows, demands, vehicle_capacities, vehicle_types,
    store_cities, adhoc_fixed_cost=ADHOC_FIXED_COST
)

In [9]:
# ============== Print Solution ==============
if solution:
    print(f"Solution found! Objective (total time): {solution.ObjectiveValue()} min\n")
    routes_data = []
    for v in range(len(vehicle_capacities)):
        if not routing.IsVehicleUsed(solution, v):
            continue
        index = routing.Start(v)
        route_stops = []
        route_load = 0
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route_load += demands[node]
            time_var = time_dim.CumulVar(index)
            arr_min, arr_max = solution.Min(time_var), solution.Max(time_var)
            name = store_names[node]
            route_stops.append({"node": node, "store": name, "load_kg": route_load, "arrival": arr_min})
            index = solution.Value(routing.NextVar(index))
        route_stops.append({"node": manager.IndexToNode(index), "store": "DEPOT", "load_kg": route_load, "arrival": solution.Min(time_dim.CumulVar(index))})
        routes_data.append(route_stops)
        route_str = " -> ".join(f"{st['store']}({st['arrival']}min)" for st in route_stops)
        vtype = "Adhoc" if vehicle_types[v] != "Fixed" else "Fixed"
        print(f"Vehicle {v} ({vtype}): {route_str}")
        print(f"  Load: {route_load} kg\n")
else:
    print("No solution found. Try relaxing constraints or adding more vehicles.")

Solution found! Objective (total time): 4198 min

Vehicle 0 (Fixed): DEPOT(360min) -> AM New Chunabhatta RNC(403min) -> AM Pahari RNC(419min) -> AM Lalpur RNC(439min) -> AM Nayatoli RNC(455min) -> AM Old Argora RNC(479min) -> AM New Piper Toli RNC(496min) -> DEPOT(518min)
  Load: 1998 kg

Vehicle 1 (Fixed): DEPOT(252min) -> AM Upper Chutia RNC(360min) -> AM Chutia Main Road RNC(375min) -> AM Tatisilwai Chowk RNC(409min) -> AM Tatisilway Rd RNC(426min) -> ASM Lower Chutia(458min) -> AM Sail City RNC(486min) -> DEPOT(507min)
  Load: 1967 kg

Vehicle 2 (Fixed): DEPOT(245min) -> AM RIMS Jora Talab RNC(360min) -> AM Bargain RNC(378min) -> AM Getlatu RNC(401min) -> AM PITAMBRAM PALACE RNC(420min) -> AM Bariatu Housing RNC(441min) -> DEPOT(481min)
  Load: 1924 kg

Vehicle 3 (Fixed): DEPOT(41min) -> AM Manifit Rd Kalimali JSR(360min) -> AM TELCO NEW JSR(384min) -> ASM Chhota Govindpur JD(407min) -> AM Shiv Mandir Rd Parsudih JSR(431min) -> AM New Parsudih JSR(447min) -> AM Dindly Basti Rd Adit

In [13]:
TARGET_DATE

datetime.date(2026, 1, 22)

In [14]:
demand_df[demand_df["delivery_date"] == "2026-01-22"]

,delivery_date,store_name,warehouse,Weight (in kg),del_time_slot,city,updated_time,start_time,end_time
0,2026-01-22,AM New Piper Toli RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,146.6220,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
1,2026-01-22,AM Ratu Rd Kamre RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,95.2830,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
2,2026-01-22,ASM Imli Chowk Harmu,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,160.0350,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
3,2026-01-22,AM Ravi Steel RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,252.5230,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
4,2026-01-22,AM New Itki Rd ITI Bus Stand RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,155.1730,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
...,...,...,...,...,...,...,...,...,...
68,2026-01-22,AM Hindpiri RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,273.3780,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
69,2026-01-22,ASM AG Colony Kadru,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,240.9800,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
70,2026-01-22,AM Pahari RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,541.9002,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00
71,2026-01-22,AM New Devi Mandap RNC,FLEET LABS TECHNOLOGIES PRIVATE LIMITED,351.2680,6 AM - 2 PM,Ranchi,06:00 - 14:00,1900-01-01 06:00:00,1900-01-01 14:00:00


In [10]:
# ============== Actual vs Optimized Adhoc Vehicles ==============
# Actual: from historical dispatch records
# Option 1: Set manually below
# Option 2: Add sheet "Actual Dispatch" with columns [delivery_date, adhoc_vehicles_used]
ACTUAL_ADHOC_USED = None  # Set to int if you have records, e.g. 3
try:
    actual_df = pd.read_excel("./data/Logistic Details_JH_Last 15 Days.xlsx", sheet_name="Actual Dispatch")
    match = actual_df[actual_df["delivery_date"].dt.date == TARGET_DATE]
    if not match.empty and ACTUAL_ADHOC_USED is None:
        ACTUAL_ADHOC_USED = int(match.iloc[0]["adhoc_vehicles_used"])
except Exception:
    pass

# Optimized: from VRP solution
optimized_adhoc_used = 0
if solution:
    for v in range(len(vehicle_capacities)):
        if routing.IsVehicleUsed(solution, v) and vehicle_types[v] != "Fixed":
            optimized_adhoc_used += 1

# Comparison
comparison = pd.DataFrame({
    "Metric": ["Adhoc vehicles used"],
    "Actual (historical)": [ACTUAL_ADHOC_USED if ACTUAL_ADHOC_USED is not None else "N/A"],
    "Optimized (VRP)": [optimized_adhoc_used]
})
print(f"Target date: {TARGET_DATE}")
print(f"\nAdhoc vehicles - Actual: {ACTUAL_ADHOC_USED if ACTUAL_ADHOC_USED is not None else 'N/A (set ACTUAL_ADHOC_USED from dispatch records)'}")
print(f"Adhoc vehicles - Optimized: {optimized_adhoc_used}")
if ACTUAL_ADHOC_USED is not None:
    diff = optimized_adhoc_used - ACTUAL_ADHOC_USED
    print(f"Difference: {diff:+d} (optimization {'reduces' if diff < 0 else 'increases'} adhoc usage by {abs(diff)})")

Target date: 2026-01-22

Adhoc vehicles - Actual: N/A (set ACTUAL_ADHOC_USED from dispatch records)
Adhoc vehicles - Optimized: 0


In [11]:
# ============== Routes Summary DataFrame ==============
if solution:
    rows = []
    for v, route in enumerate(routes_data):
        for stop in route:
            if stop["store"] != "DEPOT":
                rows.append({
                    "vehicle": v,
                    "store": stop["store"],
                    "arrival_min": stop["arrival"],
                    "arrival_time": f"{stop['arrival']//60:02d}:{stop['arrival']%60:02d}",
                    "load_kg": stop["load_kg"]
                })
    routes_summary = pd.DataFrame(rows)
    try:
        display(routes_summary)
    except NameError:
        print(routes_summary.to_string())

NameError: name 'solution' is not defined

In [12]:
# ============== Optional: Plot Routes ==============
if solution and "routes_data" in dir():
    fig, ax = plt.subplots(figsize=(10, 8))
    colors = plt.cm.tab10.colors
    # Plot depot
    ax.scatter(wh_coords[1], wh_coords[0], c="black", s=150, marker="s", label="Depot", zorder=5)
    # Plot stores
    for i, (lat, lon) in enumerate(locations[1:], 1):
        ax.scatter(lon, lat, c="gray", s=50, alpha=0.7, zorder=3)
    # Plot routes
    for v, route in enumerate(routes_data):
        lats, lons = [], []
        for stop in route:
            node = stop["node"]
            lats.append(locations[node][0])
            lons.append(locations[node][1])
        ax.plot(lons, lats, "-", color=colors[v % len(colors)], linewidth=2, label=f"Vehicle {v}", alpha=0.8)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(f"Delivery Routes - {TARGET_DATE}")
    ax.legend(loc="upper left", fontsize=8)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

NameError: name 'solution' is not defined